
<h1 style="border: 2px solid black; padding: 15px; border-radius: 12px;" align='center'>Cours IA et Applications</h1>    

<h2 align='center'> Deep Learning : Rappels sur les réseaux Fully Connected </h2>

<h3 align='center'> Jordy Palafox </h3>
<h3 align='center'> Ing2 GSI/MI - 2023/2024 </h3>
      
      
<div style="display:flex"> 
    <img src="cytech.png", style="width:250px;height:50"> 
    <img src="cy.jpg", style="width:300px;height:100px"> 
</div> 

Un point sur matériel et logiciel :

+ Vous disposez des ordinateurs de l'école avec un Processeur Intel mais **sans GPU**,

+ Avec quoi coder en Python pour du Deep Learning :
    - En **Script** : *fichier.py *puis on compile *python fichier.py* (le plus fréquent dans la vraie vie pour la mise en production,
    - En **Notebook Jupyter**: *notebook.ipynb* pour réaliser des tests,
    - Avec **Google colab** => pas de problème de ressources si on utilise pas les GPU/TPU (pour des tests sans problème de version)
    
+ En local, on peut utiliser les distributions/logiciels **Jupyter**, **VSCode** ou encore **Anaconda**.

Dans le cours, vous utiliserez **Google Colab** pour la simplicité :
+ Pas de problème de mémoire avec les environnements
+ Pas de problème de compatibilité de version !


# Quelques rappels sur Tensorflow, Keras et le calcul sur GPU

Importer les différentes libraires : 
   - *tensorflow* pour faire du deep learning https://www.tensorflow.org/?hl=fr
   - *keras* qui se situe dans tensorflow https://keras.io/ créée par François Chollet (ingénieur chez Google), particulièrement accessible et documentée
   - *numpy* pour le calcul matriciel
   - *pandas* pour manipuler des dataframes
   - *pyplot* de matplotlib pour faire de la visualisation
   
**Remarque** : il y a souvent beaucoup de messages quand on charge tensorflow (des warnings, qui ne sont pas forcément des erreurs !

In [ ]:
import tensorflow as tf 

On pensera à vérifier la version de tensorflow et keras *xxxx.\_\_version\_\_*

Cela peut permettre de vérifier si les versions sont compatibles par ailleurs.

Aussi, vérifier si une GPU est indisponible ou non pour faire les calculs. 

In [ ]:
print("Tensorflow GPU version :")
print('Keras version :')


print('GPUs available :', len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Testons sur une GPU est accessible 

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        #Currenlty memory growth needs to be the same across GPUs
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical devices", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

Les **cartes graphiques** (ou **GPU** : Graphics Processing Unit) ont permis l'essor du Deep Learning avec une grande accessibilité à "madame/monsieur Tout le monde" 
        ![image](evolution_nombre_parametre.png)
        Source : [Fournarakis] "*A Practical Guide to Neural Network Quantization*" voir : https://cms.tinyml.org/wp-content/uploads/industry-news/tinyML_Talks-_Marios_Fournarakis_210929.pdf
        
En effet, les GPU/TPU permettent **d'accèlerer grandement la vitesse de calculs en les parallélisant** (voir Deep Learning option IA en Ing3 :) ).

Avec celles-ci il y a d'autres problèmes posés (métaux rares, consommation énergétique etc).

# Chargement et exploration du dataset MNIST

On va voir maintenant comment implémenter un Réseau de Neuronnes FC en Python 
sur un jeu de données classique : *MNIST*, qui sont des images de chiffres écrits à la main.
https://keras.io/api/datasets/mnist/

*Remarque* : on peut directement charger sous la forme d'un dataset d'entrainement et un de test.

In [ ]:
# Charger le jeu de données mnist en une partie d'entraînement et une partie de test







Voyons la répartition train/test et les formes des images: 

In [ ]:
print(x_train.shape)

In [ ]:
print('')
print(f'Il y a { ...... } images d\'entrainement et { ...... } de test.')
print('--'*20)
print('Les images sont au format {} donc {} pixels par {} pixels.'.format(...... ,
   ...... , ...... ))

Regardons le type de l'image 5485 :

C'est un type de numpy, c'est le type le commun pour tensorflow et keras, d'autres librairies de deep learning nécessite des types particuliers comme Pytorch (en soit ça ne change pas grand chose et on peut les conversions entre les différents types)

In [ ]:
# Jetons un coup d'oeil aux données avec la première image et son étiquette (label) associée :

x_train[0][0:5]
# Il n'y a que des valeurs 0 donc essayons plus loin : 

x_train[0][10:15]
# voilà, les valeurs correspondent au niveau de gris : les valeurs dans les images sont entre 0 et 255 !

## Remarque sur les tenseurs

Une image peut être vue comme un **tenseur** qui est une généralisation de la notion de matrice en dimensions supérieures.
En dimension 2, un tenseur est donc une matrice (que l'on peut imaginer comme un rectangle).
En dimension 3, un tenseur est une matrice avec une "profondeur".

Une image en niveau de gris va donc être un tenseur en dimension 3 : *largeur x hauteur x 1* où la dernière dimension correspond au *<span style="color:grey">niveau de gris</span>* c'est donc comme on l'a vu une simple matrice telle que :
$$ coef \in [0,255] $$ 

Alors qu'une image en couleur (RGB) sera un tenseur en dimension 5 :
*largeur x hauteur x 3*
avec :
+ un canal pour le <span style="color:red">niveau de rouge</span>, 
+ un canal pour le <span style="color:green">niveau de vert</span>,
+ un canal pour le <span style="color:blue">niveau de bleu</span>).  

![image](rgb.png)

C'est donc un objet représenté par trois matrices supposées ou encore chaque pixel est un vecteur de longueur 3 représentant les différents niveaux de couleurs.


Affichons maintenant quelques images du dataset :

In [ ]:
images_to_display = x_train[np.random.choice(x_train.shape[0],10)]

print('Exemples d\'images du dataset Mnist'.center(50))
print('========================'.center(50))

_, axes = plt.subplots(nrows=1, ncols=10, figsize=(10, 3))
for ax, image, label in zip(axes, x_train, y_train):
    i=0
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.text(i+1,3,'--'*5)
    ax.text(i+5, 0,f'Label : {label}')
    i+=1

# Prétraitement des images

<div style="background-color: #FCF550; border-radius: 10px; border: 5px solid  orange; padding: 5px;height: 100px; margin: auto;">
    On va réaliser maintenant du <strong>préprocessing</strong>.

Sur des images, un classique est de diviser par la valeur max pour **mettre à l'échelle**.
Ainsi toutes les valeurs sont entre **0 et 1**.</div>

In [ ]:
x_train = x_train / ... .
x_test = x_test /... .

x_train[0][10]

De plus, les réseaux FC ne prennent pas en entrée que des tenseurs de dimension 1, c'est-à-dire des **vecteurs**.

Pour cela, on va transformer l'image sous forme de matrice de taille (28,28) par un 
vecteur de longueur 28x28 à l'aide de la fonction reshape :

In [ ]:
x_train = ...
x_test = ...

Il faut maintenant pré-traiter les labels car on traite un problème de **classification**
et non de **régression** hors les vecteurs y_train et y_test sont remplies de valeurs numériques
et non de modalités.

On a donc : 


In [ ]:
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

y_train[0]

Chaque label devient un vecteur avec un 1 à la position correspondante au chiffre/label, c'est ni plus ni moins que du one-hot encoding !

# Création d'un modèle simple

On va maintenant définir un modèle simple sous trois formes différentes :

Commencons par le mode Sequentiel.

Il suffit de définir un objet (le modèle) comme Sequentiel et d'ajouter les couches avec les paramètres voulus.

In [ ]:
# de keras importer layers et Sequential

...

# Créer un modèle avec Sequential :
# le première couche doit avoir 32 unités et l'activation de votre choix, n'oubliez pas de préciser la taille de l'entrée,
# une deuxième couche cachée avec 64 unités ou neurones,
# la couche de sortie doit avoir 10 sorties (une par classe), bien choisir la fonction d'activation

model = Sequential()
...
...


In [ ]:
# Avec summary vérifier l'architecture du modèle 

model. ...

# Quel est le nombre de paramètres ? 

# Retrouvez-ce nombre à la main !

La modèle a 27882 paramètres, tous "entraînables", i.e pourront être mis à jour par l'optimiseur.

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='model1_plot.png', show_shapes=True, show_layer_names=True)

<div style="background-color: #FCF550; border-radius: 10px; border: 5px solid  orange; padding: 5px;height: auto; margin: auto;">
    En résumé ... </div>

![image](network1.jpg)

# Compilation du modèle
Permet la configuration du modèle, c'est-à-dire choisir :
+ Un **optimiseur** : comment on réalise la descente de gradient, 
+ Une **fonction de perte** que l'on cherche à minimiser,
+ Une **métrique d'évaluation** du modèle.

In [ ]:
# Compiler à l'aide de compile avec un optimiseur "sgd" (descente de gradient stochastique), une perte de crossentropy 
#(attention, on fait de la classification)
# la métrique sera l'accuracy

model.compile(optimizer='...', loss='...', metrics=['...'])

# Entrainement du modèle (version simple, on verra plus compliqué plus tard)

On va entraîner le modèle avec *.fit* et préciser un certain nombre d'hyperparamètres :

+ les données x_train et y_train,
+ la taille des batchs : en général une puissance de 2, moins on a de puissance, plus on doit réduire la taille,
+ le nombre d'epochs, i.e le nombre de fois où l'on applique la descente de gradient sur l'ensemble du réseau, 
+ le pourcentage de données d'entrainement qui servent à valider.

Il y en a plein d'autres ...

In [ ]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=.1)


# Evaluation du modèle

In [ ]:
# réaliser l'évaluation du modèle sur les données de test avec ".evaluate"


loss, accuracy = model. ......

# Sortie graphique

Il est courant de représenter graphiquement la perte et la métrique d'évaluation pour comprendre comment se passe l'entrainement. 

Il peut y avoir des décrochages de l'accuracy sur les données d'entraînement, ce qui traduit d'un surapprentissage ...

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.axhline(accuracy, color='red')
plt.legend(['Training', 'Validation', 'Test'], loc='best')
plt.show()

print(f'Sur les données de test, le modèle obtient une perte de {loss} et une accuracy de {accuracy}.')


# Retour sur l'architecture et les poids d'un réseau FC

Pour bien comprendre l'architecture d'un réseau, on peut aussi regarder les poids et biais.

Remettons-nous en mémoire sur un exemple simple comment ça marche :
    
![image](simple_network.png)

In [ ]:
print('===== Coefficients du réseau ====='.center(50))
print('=== Couche d\'entrée ===')
print(f'Les deux premières lignes de la matrice de poids \
de la couche d\'entrée est de la forme : \n {model.weights[0][:2]}')
print(f"C'est une matrice de taille : {model.weights[0].shape}")
print('  =====================  ')
print(f'Le vecteur de biais est de la forme : \n{model.weights[1]}')



In [ ]:
print('===== Coefficients du réseau ====='.center(50))
print('=== Couche d\'entrée ===')
print(f'Les deux premières lignes de la matrice de poids \
             de la couche d\'entrée est de la forme : \n {model.weights[2][:2]}')
print(f"C'est une matrice de taille : {model.weights[2].shape}")
print('  =====================  ')
print(f'Le vecteur de biais est de la forme : \n{model.weights[3]}')



# Vers des modèles plus compliqués 

Cette partie est optionnelle, mais fait partie des bonnes pratiques. 
Les futurs option IA le reverront.



<div style="background-color: #FCF550; border-radius: 10px; border: 5px solid  orange; padding: 5px;height: auto; margin: auto;">
    On va introduire un autre modèle et voir comment le nombre de paramètres évolue. 

De plus on va utiliser l'API Fonctionelle de Keras, i.e manipuler les tenseur et les couches comme des fonctions de tenseurs qui renvoient des tenseurs. </div>

In [ ]:
from keras.models import Model
from keras import Input

In [ ]:
# Exemple :

input_tensor = Input(shape=(784,))
x = layers.Dense(1000, activation='tanh')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

# On définit alors le modèle avec la class Model

model_functional = Model(input_tensor, output_tensor)



Cette façon de coder le modèle est notamment très utile quand il y a plusieurs entrées et/ou sorties au modèle.

Regardons le modèle :

In [ ]:
model_functional.summary()

<div style="background-color: #FCF550; border-radius: 10px; border: 5px solid  red; padding: 5px;height: auto; margin: auto;">
    <strong>Le nombre de paramètres explose !</strong>

Faisons l'entraînement avec d'autres hyperparamètres. Cela permettra de voir au passage si l'on obtient de meilleures performances avec 28 fois plus de paramètres ! </div>



In [ ]:
model_functional.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

On va aussi en profiter pour introduire des callbacks.
Ne vous inquiètez pas, c'est des techniques plus ou moins avancées pour vous montrer ce que Keras permet de faire.   
**Vous ne serez pas évaluer là dessus.**

+ Le **EarlyStopping** interrompt l'entraînement si les performences du modèle n'augmente plus sur la métrique *monitor* (ici acc) au bout de *patience* (ici 10) epochs,

+ **ReduceLROnPlateau** surveille la perte du modèle sur l'ensemble de validation en multipliant le taux d'apprentissage (learning rate, lr) par *factor*=0.1 au bout de *patience*=5 époques s'il n'y pas d'améliorations pendant l'entraînement.

In [ ]:
callbackslist=[keras.callbacks.EarlyStopping(monitor='accuracy', patience=10),
               keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)]

In [ ]:
history = model_functional.fit(x_train, y_train, epochs=30, batch_size=16, callbacks=callbackslist, validation_split=0.1)

In [ ]:
loss, accuracy  = model_functional.evaluate(x_test, y_test)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.axhline(accuracy, color='red')
plt.legend(['Training', 'Validation', 'Test'], loc='best')
plt.show()

print(f'Sur les données de test, le modèle obtient une perte de {loss} et une accuracy de {accuracy}.')

Comparer avec le modèle simple. Quelle analyse pouvez-vous faire ? 